# Degeneracy Hunter Examples
<br/> <br/>
This notebook shows how to use the following Degeneracy Hunter features through several motivating examples:
* Add features here

## Conventions
The typographic conventions used in this tutorial are as follows.

Names of files will be in "double quotes", new terms will be in *italics*, and keywords and source code snippets will be in `fixed-width type`.

Sections that offer additional detail that may be skipped are set off in blocks like this:
<div class="alert alert-block alert-info">
    &#9998; <b>Details:</b>
    Detailed information goes here...
</div>
   

##  Setup

We start by importing Pyomo and Degeneracy Hunter.

In [1]:
from pyomo.environ import *

from idaes.core.util.model_diagnostics import DegeneracyHunter

    Import ComponentSet from pyomo.common.collections.  (deprecated in 5.7.1)
    (called from /anaconda3/envs/idaes-pse/lib/python3.7/site-
    packages/pyomo/core/kernel/component_set.py:16)


## Example 1: Linear Program with Equality Constraints


### Define

We start by defining a simple optimization problem.

In [2]:
m = ConcreteModel()

m.I = Set(initialize=[i for i in range(5)])

m.x = Var(m.I,bounds=(-10,10),initialize=1.0)
m.z = Var(bounds=(-100,100), initialize=5.0)

m.con1 = Constraint(expr=m.x[0] + m.x[1] - m.x[3] >= 10)
m.con2 = Constraint(expr=m.x[0]*m.x[3] + m.x[1] >= 0)
m.con3 = Constraint(expr=m.x[4]*m.x[3] + m.x[0]*m.x[3] - m.x[4] == 0)

m.obj = Objective(expr=sum(m.x[i]**2 for i in m.I))

m.pprint()

1 Set Declarations
    I : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

2 Var Declarations
    x : Size=5, Index=I
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :   -10 :   1.0 :    10 : False : False :  Reals
          1 :   -10 :   1.0 :    10 : False : False :  Reals
          2 :   -10 :   1.0 :    10 : False : False :  Reals
          3 :   -10 :   1.0 :    10 : False : False :  Reals
          4 :   -10 :   1.0 :    10 : False : False :  Reals
    z : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  -100 :   5.0 :   100 : False : False :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2 + x[4]**2

3 Constraint Declarations
    con1 : Size=1, Index=None, Active=True
     

### Evaluate the initial point

In [3]:
# Specify Ipopt as the solver
opt = SolverFactory('ipopt')

# Specifying an iteration limit of 0 allows us to inspect the initial point
opt.options['max_iter'] = 0

# "Solving" the model with an iteration limit of 0 load the initial point and applies
# any preprocessors (e.g., enforces bounds)
opt.solve(m, tee=True)

# Create Degeneracy Hunter object
dh = DegeneracyHunter(m)

Ipopt 3.13.2: max_iter=0


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:        3
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:        7

Total number of variables............................:        5
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        5
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total num

### Identify the constraint residuals larger than 0.1

In [4]:
dh.check_residuals(tol=0.1)

 
All constraints with residuals larger than 0.1 :

count = 0 	|residual| = 9.0
con1 : Size=1, Index=None, Active=True
    Key  : Lower : Body               : Upper : Active
    None :  10.0 : x[0] + x[1] - x[3] :  +Inf :   True
variable	lower	value	upper
x[0] 		 -10 	 1.0 	 10
x[1] 		 -10 	 1.0 	 10
x[3] 		 -10 	 1.0 	 10

count = 1 	|residual| = 1.0
con3 : Size=1, Index=None, Active=True
    Key  : Lower : Body                         : Upper : Active
    None :   0.0 : x[4]*x[3] + x[0]*x[3] - x[4] :   0.0 :   True
variable	lower	value	upper
x[4] 		 -10 	 1.0 	 10
x[3] 		 -10 	 1.0 	 10
x[0] 		 -10 	 1.0 	 10


### Identify all variables within 1 of their bounds

In [5]:
dh.check_variable_bounds(tol=1.0)

 
No variables within 1.0 of their bounds.


### Solve the optimization problem

In [6]:
opt.options['max_iter'] = 50
opt.solve(m, tee=True)

Ipopt 3.13.2: max_iter=50


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:        3
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:        7

Total number of variables............................:        5
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        5
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total nu

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 3, 'Number of variables': 5, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.13.2\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.036622047424316406}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

### Check if any constraint residuals are large than 1E-14

In [7]:
dh.check_residuals(tol=1E-14)

 
All constraints with residuals larger than 1e-14 :

count = 0 	|residual| = 9.97185747309004e-08
con1 : Size=1, Index=None, Active=True
    Key  : Lower : Body               : Upper : Active
    None :  10.0 : x[0] + x[1] - x[3] :  +Inf :   True
variable	lower	value	upper
x[0] 		 -10 	 1.4516741106008322 	 10
x[1] 		 -10 	 5.061595738300216 	 10
x[3] 		 -10 	 -3.4867300513803765 	 10

count = 1 	|residual| = 7.942586144338293e-09
con2 : Size=1, Index=None, Active=True
    Key  : Lower : Body             : Upper : Active
    None :   0.0 : x[0]*x[3] + x[1] :  +Inf :   True
variable	lower	value	upper
x[1] 		 -10 	 5.061595738300216 	 10
x[0] 		 -10 	 1.4516741106008322 	 10
x[3] 		 -10 	 -3.4867300513803765 	 10

count = 2 	|residual| = 2.2426505097428162e-14
con3 : Size=1, Index=None, Active=True
    Key  : Lower : Body                         : Upper : Active
    None :   0.0 : x[4]*x[3] + x[0]*x[3] - x[4] :   0.0 :   True
variable	lower	value	upper
x[4] 		 -10 	 -1.128125759356881 	

### Identify all variables within 10$^{-5}$ of their bounds

In [8]:
dh.check_variable_bounds(tol=1E-5)

 
No variables within 1e-05 of their bounds.


### Check the rank of the constraint Jacobian at the solution

In [9]:
dh.check_rank_equality_constraints()


Checking rank of Jacobian of equality constraints...


ValueError: k must be between 1 and min(A.shape), k=0

## Example 2: Redundant Linear Constraints

### Define

In [10]:
m = ConcreteModel()

m.I = Set(initialize=[i for i in range(1,4)])

m.x = Var(m.I,bounds=(0,5),initialize=1.0)

m.con1 = Constraint(expr=m.x[1] + m.x[2] >= 1)
m.con2 = Constraint(expr=m.x[1] + m.x[2] + m.x[3] == 1)
m.con3 = Constraint(expr=m.x[2] - 2*m.x[3] <= 1)
m.con4 = Constraint(expr=m.x[1] + m.x[3] >= 1)
m.con5 = Constraint(expr=m.x[1] + m.x[2] + m.x[3] == 1)

m.obj = Objective(expr=sum(m.x[i] for i in m.I))

m.pprint()

1 Set Declarations
    I : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}

1 Var Declarations
    x : Size=3, Index=I
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   1.0 :     5 : False : False :  Reals
          2 :     0 :   1.0 :     5 : False : False :  Reals
          3 :     0 :   1.0 :     5 : False : False :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[1] + x[2] + x[3]

5 Constraint Declarations
    con1 : Size=1, Index=None, Active=True
        Key  : Lower : Body        : Upper : Active
        None :   1.0 : x[1] + x[2] :  +Inf :   True
    con2 : Size=1, Index=None, Active=True
        Key  : Lower : Body               : Upper : Active
        None :   1.0 : x[1] + x[2] + x[3] :   1.0 :   True
    con3 : Size=1, Index=None, Active=True
     

### Evaluate the initial point

In [11]:
# Specifying an iteration limit of 0 allows us to inspect the initial point
opt.options['max_iter'] = 0

# "Solving" the model with an iteration limit of 0 load the initial point and applies
# any preprocessors (e.g., enforces bounds)
opt.solve(m, tee=True)

# Create Degeneracy Hunter object
dh = DegeneracyHunter(m)

Ipopt 3.13.2: max_iter=0


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:        6
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        3
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        3
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total num

### Identify constraints with residuals greater than 1

In [12]:
dh.check_residuals(tol=0.1)

 
All constraints with residuals larger than 0.1 :

count = 0 	|residual| = 2.0
con2 : Size=1, Index=None, Active=True
    Key  : Lower : Body               : Upper : Active
    None :   1.0 : x[1] + x[2] + x[3] :   1.0 :   True
variable	lower	value	upper
x[1] 		 0 	 1.0 	 5
x[2] 		 0 	 1.0 	 5
x[3] 		 0 	 1.0 	 5

count = 1 	|residual| = 2.0
con5 : Size=1, Index=None, Active=True
    Key  : Lower : Body               : Upper : Active
    None :   1.0 : x[1] + x[2] + x[3] :   1.0 :   True
variable	lower	value	upper
x[1] 		 0 	 1.0 	 5
x[2] 		 0 	 1.0 	 5
x[3] 		 0 	 1.0 	 5


### Solve the optimization problem and extract the solution

In [13]:
opt.options['max_iter'] = 50
opt.solve(m, tee=True)

for i in m.I:
    print("x[",i,"]=",m.x[i]())

Ipopt 3.13.2: max_iter=50


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:        6
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        3
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        3
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total nu

### Check the rank of the Jacobian of the equality constraints

In [14]:
dh.check_rank_equality_constraints()


Checking rank of Jacobian of equality constraints...
Smallest singular value(s):
0.000E+00


### Identify candidate degenerate constraints

In [15]:
ds = dh.find_candidate_equations(verbose=True,tee=True)

*** Searching for a Single Degenerate Set ***
Building MILP model...
Solving MILP model...
Using license file /Users/adowling/gurobi.lic
Academic license - for non-commercial use only
Read LP format model from file /var/folders/xy/24xvnyss36v3d8mw68tygxdw0000gp/T/tmpb69f7vyh.pyomo.lp
Reading time = 0.00 seconds
x9: 17 rows, 9 columns, 35 nonzeros
Changed value of parameter NumericFocus to 3
   Prev: 0  Min: 0  Max: 3  Default: 0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 17 rows, 9 columns and 35 nonzeros
Model fingerprint: 0x5add6450
Variable types: 5 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [2e-05, 1e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+05]
  RHS range        [1e-05, 1e+00]
Presolve removed 10 rows and 3 columns
Presolve time: 0.01s
Presolved: 7 rows, 6 columns, 16 nonzeros
Variable types: 2 continuous, 4 integer (4 binary)
Found heuristic solution: objective 0.0000200

Root relaxatio

### Find irreducible degenerate sets (IDS)

In [16]:
ids = dh.find_irreducible_degenerate_sets(verbose=True)

*** Searching for Irreducible Degenerate Sets ***
Building MILP model...
Solving MILP 1 of 2 ...
Solving MILP 2 of 2 ...

Irreducible Degenerate Set 0
nu	Constraint Name
1.0 	 con2
-1.0 	 con5

Irreducible Degenerate Set 1
nu	Constraint Name
-1.0 	 con2
1.0 	 con5
